<a href="https://colab.research.google.com/github/OlajideFemi/Carbon-Footprint/blob/main/Mixture_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Inputs: V (N x D with values {0,1} and np.nan for missing), H=2
# Outputs: pi (H,), theta (H x D), responsibilities R (N x H)

init pi = np.ones(H)/H
init theta = rng.uniform(0.25, 0.75, size=(H, D))

for iter in range(max_iters):
    # E-step
    logR = np.log(pi)[None, :]  # (1 x H) broadcast to (N x H)
    for k in range(H):
        # add sum over observed dims: v*log(theta) + (1-v)*log(1-theta)
        term = 0
        for d in range(D):
            obs = ~np.isnan(V[:, d])
            v = V[obs, d]
            loglik = v*np.log(theta[k, d]) + (1 - v)*np.log(1 - theta[k, d])
            logR[obs, k] += loglik
    # normalize
    R = softmax(logR, axis=1)

    # M-step
    Nk = R.sum(axis=0)              # (H,)
    pi = Nk / N
    for k in range(H):
        for d in range(D):
            obs = ~np.isnan(V[:, d])
            num = (R[obs, k] * V[obs, d]).sum()
            den = (R[obs, k]).sum()
            if den > 0:
                theta[k, d] = num / den
            # else: keep theta[k,d] or apply prior
